# 1 Introduction
First, we have to activate the environment. The first run may take a while as external libraries will be downloaded and compiled.

In [1]:
Pkg.activate(".")

  Activating project at `~/code/OpenScience21.jl`


# 2. The Algorithm
In this notebook, I will describe the training process on a real device and a noisy simulator. Technically the algorithm is the same as in the notebook [04_algorithm_evaluation](04_algorithm_evaluation.ipynb) except for the use of the error mitigation technic which was not necessary on the simulator written in Julia. 

We will use here the codes described in the notebooks:
* [01_solution](01_solution.ipynb)
* [03_trotter_step](03_trotter_step.ipynb)
* [04_algorithm_evaluation](04_algorithm_evaluation.ipynb)
Therefore in the beginning we import the script with includes all necessary functions.

In [2]:
# The init file contains useful scripts and imports as well as connections to the IBM Qiskit account.
include("src/Challenge/init.jl")

evaluate (generic function with 1 method)

First, we have to find the parameter for the U4 gate in trotter steps (see [03_trotter_step](03_trotter_step.ipynb)), at the same time we generate the whole circuit with ten trotter steps, this is our goal to achieve.

In [3]:
t = π
trotter_steps = 10
qc_full, params, params2, params3 = generate_circuit(trotter_steps, trotter_steps, t, init=true)
addMeasuresOS(qc_full)
# Execute simulation
check_simulation_err(qc_full, t)
qc_full

                                                                            »
 q_0: ──────────────────────────────────────────────────────────────────────»
      ┌──────────────────────────────┐                                 ┌───┐»
 q_1: ┤ U3(1.4078e-06,2.4775,3.4032) ├─────────────────────────────────┤ X ├»
      └──────────────────────────────┘                                 └─┬─┘»
 q_2: ───────────────────────────────────────────────────────────────────┼──»
                   ┌───┐                    ┌────────────────────┐       │  »
 q_3: ─────────────┤ X ├────────────────────┤ U3(π,-1.3417,6.11) ├───────■──»
                   └───┘                    └────────────────────┘          »
 q_4: ──────────────────────────────────────────────────────────────────────»
                   ┌───┐              ┌───────────────────────────────┐     »
 q_5: ─────────────┤ X ├──────────────┤ U3(4.7938e-05,0.15858,2.4322) ├─────»
                   └───┘              └─────────────────────────

The below method compares the results of the execution of two circuits.

In [4]:
function check_circuits_err(qc1, qc2)
    sym1 = execute(backend, qc1)
    sym2 = execute(backend, qc2)

    sum(abs.(sym1 - sym2))
end

check_circuits_err (generic function with 1 method)

As was described in [01_solution](01_solution.ipynb) we have to iterate 5 times the QML algorithm to find the best ansact parameter. So let's begin :)

# 3. Step 1

In [5]:
const st = 2
step1_check_qc, _, _, _ = generate_circuit(trotter_steps, 8, t, params, params2, params3, init=false)
step1_step_qc_exp, _, _, _ = generate_circuit(trotter_steps, st, t, params, params2, params3, init=true)
step1_step_qc_exp

                                                                           »
q_0: ──────────────────────────────────────────────────────────────────────»
     ┌──────────────────────────────┐                                 ┌───┐»
q_1: ┤ U3(1.4078e-06,2.4775,3.4032) ├─────────────────────────────────┤ X ├»
     └──────────────────────────────┘                                 └─┬─┘»
q_2: ───────────────────────────────────────────────────────────────────┼──»
                  ┌───┐                    ┌────────────────────┐       │  »
q_3: ─────────────┤ X ├────────────────────┤ U3(π,-1.3417,6.11) ├───────■──»
                  └───┘                    └────────────────────┘          »
q_4: ──────────────────────────────────────────────────────────────────────»
                  ┌───┐              ┌───────────────────────────────┐     »
q_5: ─────────────┤ X ├──────────────┤ U3(4.7938e-05,0.15858,2.4322) ├─────»
                  └───┘              └───────────────────────────────┘     »

We have to define our step ansact.

In [6]:
step1_ansact = generate_ansact()

                                                                   »
q_0: ──────────────────────────────────────────────────────────────»
     ┌────────────────────────────┐                                »
q_1: ┤ U3(5.0888,3.4908,0.094319) ├──────────────────■─────────────»
     └────────────────────────────┘                  │             »
q_2: ────────────────────────────────────────────────┼─────────────»
     ┌────────────────────────────┐                ┌─┴─┐           »
q_3: ┤ U3(5.0888,3.4908,0.094319) ├──■─────────────┤ X ├───────────»
     └────────────────────────────┘  │             └───┘           »
q_4: ────────────────────────────────┼─────────────────────────────»
     ┌────────────────────────────┐┌─┴─┐┌─────────────────────────┐»
q_5: ┤ U3(5.0888,3.4908,0.094319) ├┤ X ├┤ U3(3.7829,3.5114,0.755) ├»
     └────────────────────────────┘└───┘└─────────────────────────┘»
q_6: ──────────────────────────────────────────────────────────────»
                                  

In the definition of the start parameter, we can do this twofold, first, we can rand it, the second, we can use the previously found parameter (for instance if we break the optimization process).

In [7]:
#step1_start_params = [6.490463894513841, 3.434943430067894, 5.375354561541277, 4.7906997798464115, 2.849325024133124, 2.7673620360366824, 1.4940499436354768, 4.487752591524359, 3.2948988322626382, 2.0225225509500486, -1.2343170007923738, 1.0431091490261049, 1.8837296276360387, 1.2169677999557502, 5.940807695305212, 5.421210358780618, 0.3173688878579456, -6.965567540712155, 4.835967484868803, 3.0127993733543517, 4.4620544822135475, 1.378337396849606, 1.980959799643362, 3.6789144092845376, 5.417860886313657, 1.8687391611628605, 5.092938820960128]
step1_start_params = getRandParameters(step1_ansact)
nothing

Let's generate circuit to optimization.

In [8]:
# Set parameters
setparameters!(step1_ansact, step1_start_params)

# Generate step qc
step1_qc = generate_empty_circuit()
# append
append!(step1_qc, step1_step_qc_exp)
append!(step1_qc, step1_ansact)
# Add measures
addMeasuresOS(step1_qc)
step1_qc

                                                                            »
 q_0: ──────────────────────────────────────────────────────────────────────»
      ┌──────────────────────────────┐                                 ┌───┐»
 q_1: ┤ U3(1.4078e-06,2.4775,3.4032) ├─────────────────────────────────┤ X ├»
      └──────────────────────────────┘                                 └─┬─┘»
 q_2: ───────────────────────────────────────────────────────────────────┼──»
                   ┌───┐                    ┌────────────────────┐       │  »
 q_3: ─────────────┤ X ├────────────────────┤ U3(π,-1.3417,6.11) ├───────■──»
                   └───┘                    └────────────────────┘          »
 q_4: ──────────────────────────────────────────────────────────────────────»
                   ┌───┐              ┌───────────────────────────────┐     »
 q_5: ─────────────┤ X ├──────────────┤ U3(4.7938e-05,0.15858,2.4322) ├─────»
                   └───┘              └─────────────────────────

Now, we have to decide which device we will use in the notebook: Julia Simulator, Noisy simulator, or a real device.

In [9]:
@enum ExecutePlace NoisySim RealDevice JuliaSim
#useExecutePlace = NoisySim
useExecutePlace = JuliaSim
#useExecutePlace = RealDevice

JuliaSim::ExecutePlace = 2

Let's define some helper functions. One of the arguments passed to gradient descent method is Optimization Function, we have two types of these functions. One is dedicated to Julia simulator where we can use Zygote to calculate derivative, the secound one use parametr shift rules (and we run the whole circuit multiple times to mitigate the stochastic error).

In [10]:
# 
function create_julia_op(step_qc)
    step_loss(params) = loss_expected_zero_state(execute(backend, step_qc, params))
    step_dloss(params) = real(step_loss'(params))
    step_of = OptimizationFunction(false, (x) -> (step_loss(x), step_dloss(x)), step_loss)
    
    return step_of
end

function call_N_qderivative(step_qc, use_backend, x, N)
    y  = 0.0
    dy = zeros(27)
    for i in 1:N
        tmp_y, tmp_dy = qderivative(use_backend, step_qc, loss_expected_zero_state, x)
        y = y + tmp_y
        dy = dy + tmp_dy
    end
    return y/N, dy/N
end

function call_N_loss_expected_zero_state(step_qc, use_backend, x, N)
    y  = 0.0
    for i in 1:N
        tmp_y = qexecute(use_backend, step_qc, loss_expected_zero_state, x)
        y = y + tmp_y
    end
    return y/N
end


function create_dev_op(step_qc, use_backend)
#     step_of = OptimizationFunction(
#                     false,
#                     (x) -> qderivative(use_backend, step_qc, loss_expected_zero_state, x),
#                     (x) -> loss_expected_zero_state(execute(use_backend, setAndConvert(step_qc, x))))
    step_of = OptimizationFunction(
            false,
            (x) -> call_N_qderivative(step_qc, use_backend, x, 30),
            (x) -> call_N_loss_expected_zero_state(step_qc, use_backend, x, 30))

    return step_of
end

function create_op(use_execute_place, step_qc)
    if use_execute_place == JuliaSim   
        step_of = create_julia_op(step_qc)
    else
        if use_execute_place == NoisySim
            useBackend = qiskitBackendNoisySim
        else 
            useBackend = qiskitBackendJakarta
        end

        step_of = create_dev_op(step_qc, useBackend)
    end
    
    return step_of
end

create_op (generic function with 1 method)

In [11]:
step1_of = create_op(useExecutePlace, step1_qc)
nothing

In [12]:
val, xparams, itr = gradientDescent(step1_of, step1_start_params, α=0.01, maxItr=200,
                              argsArePeriodic=true, isExpectedZero=true, ϵ=1e-6, debug=true, useBigValInc=true)

3.511063015761353, 3.3521816009889287, 4.417544608304454, 3.0000206986664795, 5.37791992968064, 4.302210559633711, 4.140333720650437, 0.47940136077064527, 2.399075738492638, 3.176448959358504, 5.848718706465899, 1.6088474019541905, 4.799479126222303, 2.2738709304324836, 4.652447795687345, 5.779154796524375, 3.671615131596173, 6.01045747161123, 0.01942182523261238, 4.2057667390962825, 4.631432468115318, 2.419875873608716, 6.111645876540686, 2.4440182664126504, 0.7791835921234573, 1.1214380984040069, 4.239334999579106
Start iteration 1, value: 4.0277669767380955, best: 4.0277669767380955, dval: 3.5610184174832678
3.5142051610684333, 3.345805086113054, 4.422946596485339, 2.977714333793373, 5.378355445351012, 4.300855386221823, 4.163460544652262, 0.47142235186479486, 2.395031317178146, 3.2120591435333368, 5.855095142668443, 1.6024708870783158, 4.8069176984052575, 2.2801491628240695, 4.650517719236731, 5.7868300555025085, 3.671615131596173, 6.016833907813774, 0.019257849775306635, 4.2006373

Start iteration 28, value: 0.2728625501048178, best: 0.2130507485499535, dval: 0.12724265251757394
Big value increase: val: 0.2728625501048178, e.val: 0.2130507485499535, max(α): 21839.598126592817
Start iteration 29, value: 0.37041178924195467, best: 0.2130507485499535, dval: 0.46356889558322334
Big value increase: val: 0.37041178924195467, e.val: 0.2130507485499535, max(α): 12014.552772730103
Start iteration 30, value: 0.6195599531018594, best: 0.2130507485499535, dval: 0.5904068732443479
Big value increase: val: 0.6195599531018594, e.val: 0.2130507485499535, max(α): 6415.028933900358
Start iteration 31, value: 1.0256566031074725, best: 0.2130507485499535, dval: 1.240841321479641
Big value increase: val: 1.0256566031074725, e.val: 0.2130507485499535, max(α): 4024.1327111153773
Start iteration 32, value: 2.115987367078882, best: 0.2130507485499535, dval: 2.2180910379252055
Big value increase: val: 2.115987367078882, e.val: 0.2130507485499535, max(α): 1395.6006254830213
Start iteration

Start iteration 53, value: 0.1261566590544469, best: 0.1261566590544469, dval: 0.029170881023239767
3.5242911931471377, 2.9824893232849887, 7.4290394711911185, 1.9493795130770473, 5.219431304393027, 4.1966453220447555, 4.670703761756134, 0.2615601925296448, 2.190476126277672, 7.3732688294871345, 5.980446432155301, 1.2391551241943657, -1.8056171598485342, -3.8353636461398883, 4.788480330360732, -6.358883358977123, 3.671615131596173, 6.142185197300631, -0.16034117243169518, 3.871217368562787, 4.4252349578157295, 2.74161314036039, 6.111645876540686, -1.7378681748352636, 0.582539764222474, 1.1214380984040069, 5.73559507418305
Start iteration 54, value: 0.12552037225839946, best: 0.12552037225839946, dval: 0.02729349173485285
3.524291193141437, 2.9737791305120087, 7.40311194891313, 1.9491556635327298, 5.255974430972347, 4.198522839394898, 4.670703760877041, 0.2615589538848678, 2.1904761398246833, 7.373365249891051, 5.980446432156148, 1.2304449314197432, -1.806127997282178, -3.83932028066684

Start iteration 72, value: 0.10182960985329065, best: 0.10182960985329065, dval: 0.03831036605821768
3.5242911934416363, 2.9127479249785657, 7.06182803308474, 1.95158169928849, 5.461905220240903, 4.550573719102616, 4.67070376451911, 0.2615567285697532, 2.1904761745017383, 7.3892263309736625, 6.023241347237617, 0.9182451956879453, -1.8069858129910805, -3.908044117387343, 4.749144048853777, -6.471553207828061, 3.671615131596173, 6.1849801123829575, -0.010753141623488623, 3.862699214556773, 4.416860335183872, 2.520107157215397, 6.111645876540686, -1.7553711874510942, 0.626136156120185, 1.1214380984040069, 5.8020906858719075
Start iteration 73, value: 0.10027262549361969, best: 0.10027262549361969, dval: 0.07460920145546149
Start iteration 74, value: 1.2391168206525423, best: 0.10027262549361969, dval: 1.5206562354155522
Big value increase: val: 1.2391168206525423, e.val: 0.10027262549361969, max(α): 73.6827208379171
3.5242912007405076, 2.912744902798177, 7.071200202237823, 1.9541643512587

Start iteration 92, value: 0.04488494194439983, best: 0.04488494194439983, dval: 0.04646651397559777
3.524286632588683, 2.913318066806677, 7.071830460539047, 1.9591440874527262, 6.106827107911059, 4.490664227226794, 4.670703908973808, 0.2615548319955711, 2.190476169287022, 7.380492462196243, 6.455290166271845, 1.0278999523567787, -1.8068753543038825, -4.054900162401486, 4.6814486538955675, -7.030150023675151, 3.671615131596173, 6.617028931417286, -0.25508340406692914, 3.8627309154379392, 4.41689321363993, 2.2826456387228062, 6.111645876540686, -1.596812579665257, 0.6990305783830232, 1.1214380984040069, 5.75104206832864
Start iteration 93, value: 0.04301815077553887, best: 0.04301815077553887, dval: 0.04388020362798917
3.52428627516593, 2.9131174858726836, 7.0718288449269, 1.9592204345972688, 6.130118375893117, 4.490807335902829, 4.670704395855321, 0.26155108950831246, 2.190476169113507, 7.3806567224006425, 6.448030822553267, 0.9893224398415483, -1.806875350541664, -4.061034858046618, 4

Start iteration 110, value: 0.0034681991143038528, best: 0.0034681991143038528, dval: 0.019570513124983285
Start iteration 111, value: 0.0050959769556569185, best: 0.0034681991143038528, dval: 0.04323371284929386
Big value increase: val: 0.0050959769556569185, e.val: 0.0034681991143038528, max(α): 12.761636182394975
Start iteration 112, value: 0.006723150155126584, best: 0.0034681991143038528, dval: 0.06436131338746631
Big value increase: val: 0.006723150155126584, e.val: 0.0034681991143038528, max(α): 10.799080074443797
3.5242859307399033, 2.9135404186676697, 7.046597483023691, 2.0060460132400744, 6.2552572079829964, 4.678442571718817, 4.775479644211836, -0.06693571001375659, 2.1904717773920064, 7.51386047173614, 6.33548495707175, 1.1351399724071636, -1.8018559793810907, -4.255219732093407, 4.686081418938404, -7.353448635925127, 3.671615131596173, 6.497223722217194, -0.41547687392823596, 3.8566753527417843, 4.373056179200223, 1.9441872560624076, 6.111645876540686, -1.2188394499874282,

Start iteration 130, value: 0.00021059681688720164, best: 0.00021059681688720164, dval: 0.005358021077040011
3.539808051098958, 2.9099922136056438, 7.049255183242464, 1.9755575330780681, 6.30380734179736, 4.724503914235781, 4.768489713188832, -0.08045000444619874, 2.1775308864379674, 7.535320439566919, 6.340508308216918, 1.1257841833959736, -1.7682486730668365, -4.285451684330632, 4.686073058377985, -7.388139136623759, 3.671615131596173, 6.502247073362361, -0.416511166143054, 3.8332822451682054, 4.347887993603943, 1.9185344387020689, 6.111645876540686, -1.142463166585734, 0.8061400316097616, 1.1214380984040069, 5.957632703750839
Start iteration 131, value: 0.0001882440194379408, best: 0.0001882440194379408, dval: 0.005839821532026194
Start iteration 132, value: 0.0004136375055380899, best: 0.0001882440194379408, dval: 0.01473128740233498
Big value increase: val: 0.0004136375055380899, e.val: 0.0001882440194379408, max(α): 26.36462888593607
Start iteration 133, value: 0.0001942365635156

3.5503753701823246, 2.9101434672012196, 7.049247659106665, 1.9599599141398927, 6.325434267339583, 4.734014732842726, 4.779585062000308, -0.08049291678083968, 2.1819097109165493, 7.548032438072252, 6.340877682455684, 1.1263311941677683, -1.775684175524461, -4.296547440717939, 4.686031908716811, -7.397152257438622, 3.671615131596173, 6.502616447601126, -0.41653948200993324, 3.8255863932617324, 4.346499026099706, 1.906836484267602, 6.111645876540686, -1.1367530700337776, 0.8123851224777244, 1.1214380984040069, 5.95261077603475
Start iteration 149, value: 2.370069497579489e-5, best: 2.370069497579489e-5, dval: 0.001977395070723642
3.5511872483102622, 2.91013497256133, 7.04924767000703, 1.9598360651127875, 6.3259022840347505, 4.734062631291074, 4.779584368951727, -0.08049291660596411, 2.1818532625117752, 7.551289310550261, 6.340877611041898, 1.126299601560866, -1.7745038597529441, -4.296641290703013, 4.685911261963389, -7.397152907202046, 3.671615131596173, 6.50261637618734, -0.416539620309

Start iteration 167, value: 3.482522605084326e-6, best: 3.482522605084326e-6, dval: 0.000782467758335785
Start iteration 168, value: 3.6151224291841757e-6, best: 3.482522605084326e-6, dval: 0.0006609017736190631
3.558054021335294, 2.910586178152678, 7.049247666825787, 1.958637544360896, 6.333991969423401, 4.735775267258694, 4.779594921454793, -0.07849667048841652, 2.1798213332122516, 7.54844183523595, 6.340298186276692, 1.126928383961804, -1.7764163592215416, -4.298971445002538, 4.680922830425027, -7.3971561565620325, 3.671615131596173, 6.502036951422136, -0.41771996633452235, 3.824482704869257, 4.346498834083474, 1.905673596706011, 6.111645876540686, -1.1362799112031838, 0.8143757498889025, 1.1214380984040069, 5.951685823746867
Start iteration 169, value: 2.923699730511328e-6, best: 2.923699730511328e-6, dval: 0.0004995926969530995
3.5581186283909614, 2.9106128423996087, 7.0492476668255435, 1.9585554195417358, 6.334038986098036, 4.7357770116499704, 4.779596156815385, -0.07840133729679

(8.486462549330973e-7, Number[3.558821413758093, 2.911675166691413, 0.7660623596451392, 1.9575230363250617, 6.335677515743231, 4.7357773089924295, 4.779646547704605, -0.07689416505449397, 2.1798795505633346, 1.2664489359524005  …  6.502020569451673, -0.41833884686449196, 3.823806149086179, 4.346495735535834, 1.9049046812873456, 6.111645876540686, -1.1362661942580545, 0.8141346276299941, 1.1214380984040069, 5.951882071006834], 179)

We define the function to check if we found correct solution.

In [13]:
function check_err_of_ansact(param, check_qc=nothing)
    # Generate ansact from previous step
    ansact = generate_ansact()
    setparameters!(ansact, param)
    inv_ansact = inv(ansact)
    bindparameters!(inv_ansact)

    # Generate qc
    step_qc = generate_empty_circuit()
    # append
    append!(step_qc, inv_ansact)
    if check_qc != nothing
        append!(step_qc, check_qc)
    end
    # Add measures
    addMeasuresOS(step_qc)
    
    return check_circuits_err(qc_full, step_qc)
end

check_err_of_ansact (generic function with 2 methods)

Check if step one finish in proper way.

In [14]:
check_err_of_ansact(step1_start_params, step1_check_qc)

0.00013915404660343796

Is worth seeing the code of qderivative function available in QuantumCircuits library: https://github.com/Adgnitio/QuantumCircuits.jl/blob/main/src/Execute/Execute.jl

This method calculates the loss function, and the derivative of the loss function using the parameter shift rule (_"Quantum Circuit Learning"_ https://arxiv.org/abs/1803.00745, _"Evaluating analytic gradients on quantum hardware"_ https://arxiv.org/abs/1811.11184) and additionally, it performs the measured error mitigation as described in the notebook [01_solution](01_solution.ipynb). The _Zero Noise Extrapolation_ method is not included because I haven't known about it during the implementation but I'm going to add it in the future.

The parameter for step 1 was found on the **real device**. The code used in training can be found in [sol_step1](src/Challenge/steps/sol_step1.jl) (the code works in exactly the same way as the code above but has had more logging and checks). The log of execution of the code on a real device can be found in [sol_res_step1_Jakarta_1](src/Challenge/steps/sol_res_step1_Jakarta_1.txt) and [sol_res_step1_Jakarta_2](src/Challenge/steps/sol_res_step1_Jakarta_2.txt).

It is worth noting that the printing of the solution can be used to stop the current optimization process and start it again from the previously founded the best solution. This is not necessary for the Julia simulator but in the real device (and even on the simulator) some noises can impact the calculation of derivatives. If we can execute gradient descent many times this doesn't matter because finally, gradient descent finds the proper direction but on the real device, we would like to have as few circuits run as possible.

# 4. Step 2
We do in similar way as in step 1 but we in the addition we add the inverse of the ansact with parameters from step 1.

First we generate the check circuit and the 2 trotter step circuit which will be used in steps from 2 to 5.

In [15]:
# The parameters from previous step
step1_params = step1_start_params
# The parameter wich I found on real deviece
#step1_params = [6.490463894513841, 3.434943430067894, 5.375354561541277, 4.7906997798464115, 2.849325024133124, 2.7673620360366824, 1.4940499436354768, 4.487752591524359, 3.2948988322626382, 2.0225225509500486, -1.2343170007923738, 1.0431091490261049, 1.8837296276360387, 1.2169677999557502, 5.940807695305212, 5.421210358780618, 0.3173688878579456, -6.965567540712155, 4.835967484868803, 3.0127993733543517, 4.4620544822135475, 1.378337396849606, 1.980959799643362, 3.6789144092845376, 5.417860886313657, 1.8687391611628605, 5.092938820960128]

step2_check_qc, _, _, _ = generate_circuit(trotter_steps, 6, t, params, params2, params3, init=false)
trotter_step_qc_exp, _, _, _ = generate_circuit(trotter_steps, st, t, params, params2, params3, init=false)
trotter_step_qc_exp

                                                                              »
q_0: ─────────────────────────────────────────────────────────────────────────»
      ┌──────────────────────────────┐┌───┐┌───────────┐                 ┌───┐»
q_1: ─┤ U3(1.4078e-06,2.4775,3.4032) ├┤ X ├┤ Rz(12π/5) ├──■──────────────┤ X ├»
      └──────────────────────────────┘└─┬─┘└───────────┘  │              └─┬─┘»
q_2: ───────────────────────────────────┼─────────────────┼────────────────┼──»
           ┌────────────────────┐       │   ┌──────────┐┌─┴─┐┌──────────┐  │  »
q_3: ──────┤ U3(π,-1.3417,6.11) ├───────■───┤ Ry(3π/5) ├┤ X ├┤ Ry(8π/5) ├──■──»
           └────────────────────┘           └──────────┘└───┘└──────────┘     »
q_4: ─────────────────────────────────────────────────────────────────────────»
     ┌───────────────────────────────┐                                        »
q_5: ┤ U3(4.7938e-05,0.15858,2.4322) ├────────────────────────────────────────»
     └───────────────────────────────┘  

Next, we define the helper method which create the optimization circuit for steps from 2 to 5.

In [16]:
function generate_step_ansact(prev_step_params)
    # Generate ansact from previous step
    prev_step_ansact = generate_ansact()
    setparameters!(prev_step_ansact, prev_step_params)
    prev_step_inv_ansact = inv(prev_step_ansact)
    bindparameters!(prev_step_inv_ansact)

    # Generate asnact for current step
    ansact = generate_ansact()
    start_params = getRandParameters(ansact)
    # Set parameters
    setparameters!(ansact, start_params)

    # Generate step qc
    step_qc = generate_empty_circuit()
    # append
    append!(step_qc, prev_step_inv_ansact)
    append!(step_qc, trotter_step_qc_exp)
    append!(step_qc, ansact)
    # Add measures
    addMeasuresOS(step_qc)

    return start_params, step_qc
end

generate_step_ansact (generic function with 1 method)

We generate the optimization circuit.

In [17]:
step2_start_params, step2_qc = generate_step_ansact(step1_params)
step2_qc

                                                                          »
 q_0: ────────────────────────────────────────────────────────────────────»
        ┌──────────────────────────┐                                      »
 q_1: ──┤ U3(1.114,-6.502,-3.6716) ├──────────────────────■───────────────»
        └──────────────────────────┘                      │               »
 q_2: ────────────────────────────────────────────────────┼───────────────»
       ┌────────────────────────────┐                   ┌─┴─┐             »
 q_3: ─┤ U3(-1.9049,1.1363,-6.1116) ├───■───────────────┤ X ├─────────────»
       └────────────────────────────┘   │               └───┘             »
 q_4: ──────────────────────────────────┼─────────────────────────────────»
      ┌──────────────────────────────┐┌─┴─┐┌─────────────────────────────┐»
 q_5: ┤ U3(-0.81413,-5.9519,-1.1214) ├┤ X ├┤ U3(0.41834,-4.3465,-3.8238) ├»
      └──────────────────────────────┘└───┘└─────────────────────────────┘»
 q_6: ──────

And optimization function object.

In [18]:
step2_of = create_op(useExecutePlace, step2_qc)
nothing

Now we can find the best parameters.

In [19]:
val, xparams, itr = gradientDescent(step2_of, step2_start_params, α=0.01, maxItr=200,
                              argsArePeriodic=true, isExpectedZero=true, ϵ=1e-6, debug=true, useBigValInc=true)

5.730022034267372, 3.0954841371265096, 4.642308363935754, 6.205985258973753, 4.727424036800263, 1.810806444535643, 5.457195601981444, 5.706667707995723, 6.1565570825105285, 3.152486933178031, 5.2938796132478405, 0.5345574246193956, 4.866603124854936, 5.187308475308523, 2.5620874824372075, 2.6126642763984718, 1.2889500630074973, 3.227041167713116, 0.12522034088555156, 4.730742889364116, 1.3982509897150546, 2.974960300950706, 5.1508335548441195, 1.6618292698849835, 4.715283712074324, 1.617949724057262, 4.951791693788761
Start iteration 1, value: 2.80307004788904, best: 2.80307004788904, dval: 2.5199352889096227
5.7171968442959304, 3.0983711877822127, 4.649491052656561, 6.199086094324643, 4.728017714333281, 1.8111664544487702, 5.456456077830602, 5.7017079583157, 6.14901827262559, 3.1272875802889346, 5.291018203345254, 0.5374444752750989, 4.85315520573859, 5.188556438842406, 2.562216692724674, 2.609579623767475, 1.2889500630074973, 3.22417975781053, 0.12996163339141312, 4.735867109292221, 

4.45669248338909, 2.776286516302747, 5.493582606866006, 5.755449054494228, 6.0069335229370004, 1.833035765697031, 5.138769549314229, 5.494186734949932, 6.045843712296401, 1.941960975774032, 5.54702260298233, 0.2153598037956057, 4.7235444579538965, 5.638965132261408, 2.5733208248625155, 2.4140300921973745, 1.2889500630074973, 3.4801841574480865, -0.08023310152029128, 4.843056396260334, 1.5091217788488298, 3.8821904248103722, 5.1508335548441195, 1.1951045919670924, 5.375575977255886, 1.617949724057262, 4.076712783961421
Start iteration 20, value: 0.04502664218386256, best: 0.04502664218386256, dval: 0.09678454656847651
4.456692442315214, 2.7625907433671, 5.354094141494649, 5.755445840157282, 5.986116423094888, 1.833032527481932, 5.013267853722959, 5.494189412502151, 6.045843712298518, 1.9084077213657622, 5.599334884411532, 0.20166403085995763, 4.721987382733901, 5.6496248250027215, 2.573320701947912, 2.4127156988354908, 1.2889500630074973, 3.53249643887738, -0.08024695682366166, 4.843056

Start iteration 38, value: 0.007313570536299746, best: 0.007313570536299746, dval: 0.019011989856180023
4.456692467902669, 2.790385007512949, 5.280242524868121, 5.696627059419771, 6.187437825682683, 1.8334759432321193, 5.086210508374536, 5.489152997896085, 6.045843712303966, 1.9017496752549221, 5.807499320536488, 0.230533631277302, 4.720226326193257, 5.8541679002924045, 2.510464239014887, 2.321261156988329, 1.2889500630074973, 3.7406608750026638, -0.0804547210846085, 4.843056396511422, 1.5091217813245088, 3.966411590225206, 5.1508335548441195, 1.4073935986625217, 5.499435391905937, 1.617949724057262, 4.352721972429118
Start iteration 39, value: 0.007162246750386172, best: 0.007162246750386172, dval: 0.020306787293032633
Start iteration 40, value: 0.052593489976771754, best: 0.007162246750386172, dval: 0.22018101060135747
Big value increase: val: 0.052593489976771754, e.val: 0.007162246750386172, max(α): 551.2646637280263
4.456678118080486, 2.753869116822719, 5.288339952209056, 5.695092

Start iteration 57, value: 0.00309432963157865, best: 0.0028216207300834732, dval: 0.01919386145195573
4.4391266078666245, 2.680257648174885, 5.320548613017085, 5.638395579524171, 6.440759977595858, 1.833549726761622, 5.024560316768026, 5.487958156463789, 6.0458437123039666, 1.9122674381218299, 5.86001031286016, 0.20890547541397322, 4.779241794586407, 5.860176770829848, 2.3764701125667416, 2.267295056418316, 1.2889500630074973, 3.793171867326404, -0.08046846089473701, 4.843056396635908, 1.5091217824039134, 3.9668964416136228, 5.1508335548441195, 1.3849759876466792, 5.503339095456392, 1.617949724057262, 4.3418216676202555
Start iteration 58, value: 0.0025670256148909204, best: 0.0025670256148909204, dval: 0.01830394526669578
4.435301723247849, 2.679660853562728, 5.3205526883874965, 5.638312298244125, 6.372319791227707, 1.8335420913115972, 5.0247335576829535, 5.488835624575664, 6.0458437123112105, 1.9104272570965255, 5.865428208253893, 0.20865869342009344, 4.7771989708448555, 5.860045069

Start iteration 73, value: 0.0007900971878419545, best: 0.0007900971878419545, dval: 0.006363062813700249
Start iteration 74, value: 0.0008818962791044491, best: 0.0007900971878419545, dval: 0.010540737593963248
Big value increase: val: 0.0008818962791044491, e.val: 0.0007900971878419545, max(α): 10.514672279584875
Start iteration 75, value: 0.0040989187200492685, best: 0.0007900971878419545, dval: 0.05750508110784344
Big value increase: val: 0.0040989187200492685, e.val: 0.0007900971878419545, max(α): 45.821397324625664
Start iteration 76, value: 0.020090472368432113, best: 0.0007900971878419545, dval: 0.11495332197979834
Big value increase: val: 0.020090472368432113, e.val: 0.0007900971878419545, max(α): 41.117589304805655
Start iteration 77, value: 0.003746919332071329, best: 0.0007900971878419545, dval: 0.054971997248263516
Big value increase: val: 0.003746919332071329, e.val: 0.0007900971878419545, max(α): 3.7810967883501996
Start iteration 78, value: 0.6852340160299641, best: 0.0

...

4.420259748817746, 2.583836762250109, 5.334979758674182, 5.638948449727218, 6.611033626790561, 1.8661175650543407, 4.978201575210984, 5.50401546638511, 6.045843703007436, 1.9958965276064247, 5.876139373922542, 0.18137089815273463, 4.818771585119015, 5.811837479983379, 2.1740228405746524, 2.1502028259194264, 1.2889500630074973, 3.8245484629351942, -0.06892286250815953, 4.843056439432846, 1.5091220569930828, 3.9893781074717714, 5.1508335548441195, 1.3857614308741721, 5.516109239362656, 1.617949724057262, 4.327979259589286
Start iteration 118, value: 0.00011726926248097424, best: 0.00011726926248097424, dval: 0.002654505080732991
Start iteration 119, value: 0.00013103103541491952, best: 0.00011726926248097424, dval: 0.003616064166318442
Big value increase: val: 0.00013103103541491952, e.val: 0.00011726926248097424, max(α): 8.456008749208815
4.420259748514766, 2.583840969987072, 5.3378511007023155, 5.638946996513713, 6.6135645304289294, 1.8667993280578623, 4.978149892069991, 5.502407651352

Start iteration 141, value: 5.6868295314175645e-5, best: 5.6868295314175645e-5, dval: 0.001467491919692518
4.420060226008414, 2.5838429749131278, 5.345982122047011, 5.637904481818574, 6.623980437478872, 1.8837852324157758, 4.973069683206281, 5.518524650915277, 6.045843323699275, 2.0046497133697296, 5.875994582805127, 0.1847189219846301, 4.817842033985686, 5.807485387179898, 2.1574285410860408, 2.1443718438349975, 1.2889500630074973, 3.824382198507198, -0.06890459334109347, 4.843056443834688, 1.5091221093787504, 3.9960668848018464, 5.1508335548441195, 1.396103071953724, 5.516157483074741, 1.617949724057262, 4.311915982301013
Start iteration 142, value: 5.5140223921715145e-5, best: 5.5140223921715145e-5, dval: 0.0014412326696634226
Start iteration 143, value: 5.721348128821369e-5, best: 5.5140223921715145e-5, dval: 0.0026867212176856503
Start iteration 144, value: 0.00014640124265102972, best: 5.5140223921715145e-5, dval: 0.009294100159937368
Big value increase: val: 0.000146401242651029

Start iteration 162, value: 3.2961795986790104e-5, best: 3.2961795986790104e-5, dval: 0.0008921090895758401
Start iteration 163, value: 3.419792133943068e-5, best: 3.2961795986790104e-5, dval: 0.0015602329658521291
4.4091006506040165, 2.5838431832599262, 5.34660406434643, 5.637456753041592, 6.62400063935423, 1.8889443089694309, 4.966889085034504, 5.52463267907903, 6.04580861470085, 2.0100373992972655, 5.875336693557403, 0.19041442072660586, 4.813355099953033, 5.804821882108391, 2.1542645355229673, 2.1482987388654973, 1.2889500630074973, 3.8236267406967617, -0.06890103698320932, 4.8430564438350014, 1.5091221093788223, 3.9982548496415142, 5.1508335548441195, 1.4036051259872409, 5.516156678328119, 1.617949724057262, 4.3089925400272255
Start iteration 164, value: 3.1535683159369666e-5, best: 3.1535683159369666e-5, dval: 0.0007927975054953657
Start iteration 165, value: 3.165154379467952e-5, best: 3.1535683159369666e-5, dval: 0.0011082666048206624
4.409387599085914, 2.583843184041332, 5.347

Start iteration 182, value: 2.0627687771819905e-5, best: 2.0627687771819905e-5, dval: 0.0007546123719206488
4.409090507832899, 2.583843185267295, 5.347352217839006, 5.637456838280288, 6.625533777214135, 1.8945981550918811, 4.963582839558343, 5.53071443106051, 6.045106907607716, 2.014706744304294, 5.873526611653746, 0.19439409099648056, 4.810345055039812, 5.799878513532257, 2.1506917064169673, 2.148298738867938, 1.2889500630074973, 3.821548213779607, -0.06798208072092242, 4.8430564438350014, 1.5091221093788223, 4.0008577471716595, 5.1508335548441195, 1.4036051358851136, 5.515705534108977, 1.617949724057262, 4.305598872887479
Start iteration 183, value: 1.9929463722117477e-5, best: 1.9929463722117477e-5, dval: 0.0005988932945519623
Start iteration 184, value: 2.022977798742264e-5, best: 1.9929463722117477e-5, dval: 0.0010246869209987047
4.4090862555707995, 2.5838431852708394, 5.348418015140181, 5.637456838290594, 6.626835302631014, 1.894601977118, 4.962571682986124, 5.530714332792149, 6.

(1.3346173020368763e-5, Number[4.407028387130676, 2.583843185272351, 5.349085256237609, 5.637442830036508, 6.627288219592974, 1.8952423561469347, 4.95969481321939, 5.5332011401343095, 6.042464320705847, 2.017568397910976  …  3.8215310053695486, -0.06755989143947141, 4.8430564438350014, 1.5091221093788223, 4.002893705513185, 5.1508335548441195, 1.403763932878767, 5.515621715878243, 1.617949724057262, 4.303428727211307], 200)

Check if step one finish in proper way.

In [20]:
check_err_of_ansact(step2_start_params, step2_check_qc)

0.0008184931687979157

The parameter for step 2 was found on the noisy simulator. The code used in training can be found in [sol_step2](src/Challenge/steps/sol_step2.jl) (the code works in exactly the same way as the code above but has had more logging and checks). The log of execution of the code on a noisy simulator can be found in [sol_res_step2_1](src/Challenge/steps/sol_res_step2_1.txt).


# 5. Step 3

In [21]:
# The parameters from previous step
step2_params = step2_start_params
# The parameter wich I found on real deviece
#step2_params = [6.357739083193767, 4.110466988994183, 4.566133708100635, -4.727581603064405, 6.207391247986331, -1.3477711816915519, -0.7698439582879397, 4.677177128084215, -1.88364164245155, -6.961405873009646, 2.3088656324648755, 1.9854073372986245, -2.4954433452385736, 6.326779366664302, -0.7789648567030898, 2.995616831369772, 6.237335880828537, 0.6943207146536418, 0.37027914593259564, 0.9373275950100715, 2.0325673904575456, 1.3926281522358193, 5.042496816647515, 0.3933698184667445, 3.4025273194720507, 0.9766806621264883, 3.8089690519454855]

step3_check_qc, _, _, _ = generate_circuit(trotter_steps, 4, t, params, params2, params3, init=false)

step3_start_params, step3_qc = generate_step_ansact(step2_params)
step3_of = create_op(useExecutePlace, step3_qc)

val, xparams, itr = gradientDescent(step3_of, step3_start_params, α=0.01, maxItr=200,
                              argsArePeriodic=true, isExpectedZero=true, ϵ=1e-6, debug=true, useBigValInc=true)

5.179285585642374, 4.753123351964444, 0.5215840737813875, 0.38748286290394596, 5.83661299282498, 4.956008020122005, 5.4623489980178475, 3.079628017034936, 5.415525974800484, 0.8263638925606145, 2.247979499465953, 4.677226234948385, 1.8075814262338215, 1.939953469100184, 1.9852481020379247, 1.4640336713034126, 5.740062122325872, 3.2152947500151705, 3.203865076993627, 2.253612381900416, 3.474422097546678, 1.3744080904331015, 4.627180138230332, 3.8399647902484784, 0.0021954100260988796, 1.1900773401804101, 4.935281452062187
Start iteration 1, value: 2.6216447489255805, best: 2.6216447489255805, dval: 1.4474220501174344
5.188684267193688, 4.74389749108536, 0.522274882514924, 0.37582684594881177, 5.846184576955321, 4.963027345148132, 5.453351999736433, 3.0763138311077465, 5.408007208258421, 0.8204822677553716, 2.2335052789647785, 4.668000374069302, 1.8022528242640508, 1.9346613594517161, 1.9858821500022603, 1.4742351718083757, 5.740062122325872, 3.200820529513996, 3.2070091005034644, 2.2559

Start iteration 22, value: 0.23876697165158386, best: 0.23876697165158386, dval: 0.19312706338659355
5.0711610581000155, 3.9861047200575146, 0.5242645844688463, -0.9642447098084294, 5.977818261533169, 5.068908894411822, 5.904653275483749, 3.3397474870215946, 4.173268689288687, 0.8311952886342447, 1.974088680529367, 3.841951154901133, 2.2460590569253873, 1.6243769038174398, 1.9837402493759257, 1.142202336856739, 5.740062122325872, 2.941592342360657, 3.2325852837210047, 2.0995670404841023, 3.474611315979243, 1.2867268265588727, 4.627180138230332, 4.589903958461503, -1.2687315731646485, 1.1900773401804101, 4.9352918720266015
Start iteration 23, value: 0.23623362531542383, best: 0.23623362531542383, dval: 0.22947473473711782
4.731642990189093, 3.98610158392225, 0.5312355329653698, -0.9870545426237554, 5.977818343936684, 5.069125050520955, 5.9046288222798795, 3.3395809537705063, 4.17066240304394, 1.003430188843028, 1.9748341536012595, 3.8419458068033516, 2.245963562015374, 1.606173082835567

Start iteration 41, value: 0.034697803679634376, best: 0.008850983252438252, dval: 0.17266870501516723
Big value increase: val: 0.034697803679634376, e.val: 0.008850983252438252, max(α): 17.569583012013467
4.6952308999433905, 3.9861178642552244, 0.7669340761578791, -1.550314991664236, 6.166868338584123, 5.102575471777804, 6.280459988112447, 3.243749909277739, 4.15980763484961, 1.516222318351947, 1.9897822065819226, 3.841973569895606, 1.9429049250322221, 1.6811022469509662, 2.288758409864721, 0.7106951831817916, 5.740062122325872, 2.947938973396635, 3.232584891901975, 2.220792442960576, 3.473746063878702, 1.494945261530955, 4.627180138230332, 4.534016550063963, -1.4348882001106766, 1.1900773401804101, 4.918137602743178
Start iteration 42, value: 0.004700191515204149, best: 0.004700191515204149, dval: 0.037895758593773775
4.695092728876813, 3.9861178647240703, 0.7670285592512383, -1.5738527562839209, 6.177632594519815, 5.110283674043552, 6.2812175233057665, 3.2435082965616187, 4.15968084

Start iteration 57, value: 0.0002562220197728371, best: 0.0002562220197728371, dval: 0.012745347644093241
Start iteration 58, value: 0.0009917686237074905, best: 0.0002562220197728371, dval: 0.025573537322377993
Big value increase: val: 0.0009917686237074905, e.val: 0.0002562220197728371, max(α): 18.34957156864321
4.694940915676793, 3.99119433823363, 0.791853347358708, -1.6070031364285409, 6.1964548617060045, 5.104573936445226, 6.315453497964083, 3.2404279110855203, 4.145400204757905, 1.5583312003797598, 2.012672653931153, 3.850630555575266, 1.914645876068995, 1.6921076660215688, 2.4477547825370802, 0.716504961882754, 5.740062122325872, 2.957196116953038, 3.232210157408371, 2.221662387266457, 3.473704559095135, 1.4995432355930702, 4.627180138230332, 4.619959019387529, -1.6185530359644842, 1.1900773401804101, 4.925397608875297
Start iteration 59, value: 0.00010991117575641312, best: 0.00010991117575641312, dval: 0.006119740133055243
4.694939297597917, 3.991219689105386, 0.79224370283464

Start iteration 77, value: 1.1895540762598628e-6, best: 1.1895540762598628e-6, dval: 0.0006648195134326199
4.694937526287066, 3.9986757004288984, 0.7887614485615964, -1.602222352893077, 6.195116989524687, 5.10587215078546, 6.289070999951748, 3.2403713583360823, 4.142210121063459, 1.5591353959230585, 2.015045162107819, 3.8633886324736078, 1.914579537543359, 1.6904733347411915, 2.447044258618329, 0.7168021451661278, 5.740062122325872, 2.958155584823832, 3.232205323392328, 2.223860203879877, 3.473704559146185, 1.4969973576241042, 4.627180138230332, 4.614123009380571, -1.6139319438448096, 1.1900773401804101, 4.921350151387696
Start iteration 78, value: 1.0743606720023479e-6, best: 1.0743606720023479e-6, dval: 0.000630034118881375
4.6949374380324045, 3.9987094094019917, 0.7887787951180099, -1.6031063390945763, 6.194872134860197, 5.105872169930256, 6.288994511008591, 3.240371351281508, 4.142209147202213, 1.5591600265360877, 2.0148157985129633, 3.8634461168808123, 1.914574574389089, 1.6904122

(6.863658567291087e-7, Number[4.6949374380324045, 3.9987094094019917, 0.7887787951180099, -1.6031063390945763, 6.194872134860197, 5.105872169930256, 6.288994511008591, 3.240371351281508, 4.142209147202213, 1.5591600265360877  …  2.9580628277147025, 3.2322053199723424, 2.224989189854671, 3.4737045590790148, 1.497008233928827, 4.627180138230332, 4.6141228440780155, -1.6139180808536293, 1.1900773401804101, 4.921265733990603], 79)

In [22]:
check_err_of_ansact(step3_start_params, step3_check_qc)

0.0007146200539522847

The parameter for step 3 was found on the noisy simulator. The code used in training can be found in [sol_step3](src/Challenge/steps/sol_step3.jl) (the code works in exactly the same way as the code above but has had more logging and checks). The log of execution of the code on a noisy simulator can be found in [sol_res_step3_1](src/Challenge/steps/sol_res_step3_1.txt).


# 6. Step 4

In [23]:
# The parameters from previous step
step3_params = step3_start_params
# The parameter wich I found on real deviece
#step3_params = [5.189896155542379, 4.707110397329062, 4.325905810068413, -0.8159924212917966, 3.110011220351916, 4.667709359716189, 1.0215152198560669, 5.116208100394577, 4.395063842373562, 4.370201901371649, 4.9135180721869425, 4.949975295863072, 4.637313944153916, 4.042830477313872, -0.6901894825683017, 1.1827639351923742, 3.0974447481049743, 4.022975117724089, 0.705887140029585, 3.514630564645274, 1.755049621148228, 4.151136862582433, 5.722357530716369, 0.8667811089902828, 0.3630963695624429, 0.1785816271535451, 3.4571312587872733]

step4_check_qc, _, _, _ = generate_circuit(trotter_steps, 2, t, params, params2, params3, init=false)

step4_start_params, step4_qc = generate_step_ansact(step3_params)
step4_of = create_op(useExecutePlace, step4_qc)

val, xparams, itr = gradientDescent(step4_of, step4_start_params, α=0.01, maxItr=200,
                              argsArePeriodic=true, isExpectedZero=true, ϵ=1e-6, debug=true, useBigValInc=true)

4.132952070077372, 3.8336683961520124, 3.869424599885051, 3.255532343763136, 4.819668041489996, 1.8550633499457236, 5.904734081578966, 1.9310629356854208, 2.0315727047743017, 0.9054745754380559, 4.68871899704009, 0.29066513038441505, 4.383523160019999, 2.1342514022731183, 0.6009415884262946, 5.853399229157503, 6.0306520390775855, 0.7520358101027647, 0.8833285569740971, 2.969608324035075, 4.024333226692658, 4.4945134564951035, 6.048640247836036, 2.827620671397382, 5.002757260392745, 4.3961461296906545, 3.968748059222293
Start iteration 1, value: 4.544000413899647, best: 4.544000413899647, dval: 5.072514896713679
4.131617387554877, 3.796719198513003, 3.89764188759895, 3.2491065177850857, 4.807487726900782, 1.8708682407905015, 5.881373301574726, 1.891713883161123, 1.9877356867700973, 0.9013840612236561, 4.692997063241225, 0.2537159327454056, 4.379970179977723, 2.184976551240255, 0.5699267540226287, 5.860116981969993, 6.0306520390775855, 0.7563138763038996, 0.851140476729491, 2.94722958091

Start iteration 19, value: 0.07347982907320766, best: 0.07347982907320766, dval: 0.09028631369283306
4.168580860799001, 3.111448084204181, 3.9737127582965437, 2.316843964891554, 3.5294911558020194, 2.3600885302282406, 5.829035120159986, 1.751066632722041, 1.740321788484499, 1.5254070343218722, 4.62225963108548, -0.43155518156340106, 3.832870586698605, 2.1551750102203378, 0.42189751872184034, 5.677475626239471, 6.0306520390775855, 0.6855764441481337, 0.7680370587778863, 2.889414334981731, 3.9986621976895265, 5.043450142363293, 6.048640247836036, 3.0722468696762926, 5.10919734790699, 4.3961461296906545, 4.242740513263217
Start iteration 20, value: 0.0729083051408551, best: 0.0729083051408551, dval: 0.09121489570749491
Start iteration 21, value: 0.1311780532730132, best: 0.0729083051408551, dval: 0.2700375640818763
Big value increase: val: 0.1311780532730132, e.val: 0.0729083051408551, max(α): 7.785413297070514
4.168425449094483, 3.075905741642974, 3.973690511961176, 2.3135850145959864, 3

Start iteration 37, value: 0.005561522571250858, best: 0.005561522571250858, dval: 0.018079447812540214
4.011223131388128, 3.008258826205177, 3.9805192573557844, 2.024086147132652, 3.0541700440402884, 2.3883432402164377, 5.829021713552338, 1.7667674501252704, 1.6736542847866642, 1.6454501687126466, 4.705722709761507, -0.5505024560421267, 4.0012099356906194, 2.076411327191548, 0.5005699107839526, 5.275692155995431, 6.0306520390775855, 0.7690395228241825, 0.6327348278016118, 2.8025948038574864, 3.998662057303406, 5.070897035908817, 6.048640247836036, 3.0522457717884532, 5.107940022643642, 4.3961461296906545, 4.370604767012844
Start iteration 38, value: 0.004946506012836673, best: 0.004946506012836673, dval: 0.02171750931303354
4.008318210878246, 3.0071771336540474, 3.980479942406068, 2.0330443951888446, 3.021504718437034, 2.3875183999797063, 5.828977573207109, 1.767067360380989, 1.674649113734862, 1.640062161232693, 4.71870165172137, -0.5586711471515391, 4.001228226183233, 2.076514013918

3.9678394877459366, 3.008019993907542, 3.9804684389379776, 2.001068178421993, 2.9615836191922624, 2.3884804014485312, 5.806329907907157, 1.767092301898424, 1.6132626659762455, 1.670658819693137, 4.723508896045178, -0.5609297726508883, 4.001229896067156, 2.100078867562427, 0.5751182853760826, 5.181021491768665, 6.0306520390775855, 0.7868257091130649, 0.5041241615264542, 2.8104706903888736, 3.998662608644569, 5.120341680007646, 6.048640247836036, 2.941889179497071, 5.106354307338621, 4.3961461296906545, 4.34466172393099
Start iteration 57, value: 0.00047873749615280403, best: 0.00047873749615280403, dval: 0.00778024662378049
3.9678360022895793, 3.008258586912948, 3.980468196697536, 2.0009628594618483, 2.961479348684175, 2.3884813041689945, 5.807127393163816, 1.7670924050145072, 1.6131613080380038, 1.6707009308171665, 4.723628155458722, -0.5603997198607, 4.001229889925649, 2.1109693446001097, 0.5762550446235106, 5.180886175715869, 6.0306520390775855, 0.7869449685266856, 0.5039717931831176

Start iteration 79, value: 3.2515396228630875e-5, best: 3.2515396228630875e-5, dval: 0.0029027590402181253
Start iteration 80, value: 0.006771442940113544, best: 3.2515396228630875e-5, dval: 0.07934095048690826
Big value increase: val: 0.006771442940113544, e.val: 3.2515396228630875e-5, max(α): 125.9590035546791
Start iteration 81, value: 4.402573810760034e-5, best: 3.2515396228630875e-5, dval: 0.004225987093212785
Big value increase: val: 4.402573810760034e-5, e.val: 3.2515396228630875e-5, max(α): 2.965893313823727
Start iteration 82, value: 3.381997762394252e-5, best: 3.2515396228630875e-5, dval: 0.0028969979274080515
3.9606975857450633, 3.0129485612876366, 3.9811542229946757, 1.9648242371631264, 2.9543550849313176, 2.3885258207754028, 5.739070222837574, 1.7670929911534907, 1.5824623191270688, 1.6712137534764069, 4.736229148255265, -0.5498799366678354, 3.9660974373670723, 2.110623257120409, 0.6178725629460889, 5.170325617305403, 6.0306520390775855, 0.7995459613405301, 0.5036219612491

3.957292300325639, 3.012948321358461, 3.9731752445783863, 1.9693069318609349, 2.953948144033134, 2.3886203241289965, 5.740030478949285, 1.7670921483368784, 1.5800183998430086, 1.67839920048366, 4.736206093919485, -0.5498799470311614, 3.9714572041825686, 2.113444382917182, 0.6154306093278522, 5.170548347112328, 6.0306520390775855, 0.7995229070046829, 0.5036233935902746, 2.8114694639207114, 3.998662425127379, 5.137689807790708, 6.048640247836036, 2.929185573918329, 5.106525352102298, 4.3961461296906545, 4.326450097444419
Start iteration 100, value: 9.755778993259065e-7, best: 9.755778993259065e-7, dval: 0.00029299229509990865
3.9572764819112427, 3.012948321204649, 3.9730526727175737, 1.9693468698323748, 2.9539403546498355, 2.3886222813849494, 5.740085044700873, 1.7670921054608462, 1.5800184017338357, 1.678362844836192, 4.736196745462032, -0.549879947037805, 3.971812374633678, 2.113443285104604, 0.6154157258925214, 5.170528535471352, 6.0306520390775855, 0.7995135585472093, 0.5036234049977

(8.839052720404082e-7, Number[3.9572764819112427, 3.012948321204649, 3.9730526727175737, 1.9693468698323748, 2.9539403546498355, 2.3886222813849494, 5.740085044700873, 1.7670921054608462, 1.5800184017338357, 1.678362844836192  …  0.7995135585472093, 0.503623404997783, 2.811466672985968, 3.998662394366006, 5.137689609526961, 6.048640247836036, 2.929158261214241, 5.106525351894597, 4.3961461296906545, 4.326450230711219], 100)

In [24]:
check_err_of_ansact(step4_start_params, step4_check_qc)

0.0005320614433431119

The parameter for step 4 was found on the noisy simulator. The code used in training can be found in [sol_step4](src/Challenge/steps/sol_step4.jl) (the code works in exactly the same way as the code above but has had more logging and checks). The log of execution of the code on a noisy simulator can be found in [sol_res_step4_1](src/Challenge/steps/sol_res_step4_1.txt).


# 7. Step 5

The parameter for step 5 was found on the noisy simulator. The code used in training can be found in [sol_step5](src/Challenge/steps/sol_step5.jl) (the code works in exactly the same way as the code above but has had more logging and checks). The log of execution of the code on a noisy simulator can be found in [sol_res_step5_1](src/Challenge/steps/sol_res_step5_1.txt).

In [25]:
# The parameters from previous step
step4_params = step4_start_params
# The parameter wich I found on real deviece
#step4_params = [1.1187171575015713, 0.5330092890891549, 2.586441510958004, 1.5178154678919513, 3.137963883844627, 4.228048049765462, 4.839509162353759, 2.9578752785288516, 4.768574523872119, 1.6545920069530968, 2.9732151650981935, 5.438796728232314, 1.3567895934397545, 2.2177922918734536, 2.0165705795863307, -0.14637716576453905, 0.4175300959136594, 4.311416597020989, 5.65363080884547, 5.1078093853571485, 2.6110312065705705, 3.9648509676298693, 2.989797720639175, 2.5851197335780856, 1.9933603087957705, 0.5174013121704684, 2.1454323563784383]

step5_start_params, step5_qc = generate_step_ansact(step4_params)
step5_of = create_op(useExecutePlace, step5_qc)

val, xparams, itr = gradientDescent(step5_of, step5_start_params, α=0.01, maxItr=200,
                              argsArePeriodic=true, isExpectedZero=true, ϵ=1e-6, debug=true, useBigValInc=true)

1.4877769299024328, 3.0089057075104204, 1.3871161895815551, 5.248212159411048, 5.978097509723719, 5.322640459811362, 1.4379080078908066, 2.0268335218959703, 3.8993959688906883, 4.521580326619811, 3.6849847599083647, 4.6668411369445515, 1.8876732892405772, 2.066868129016876, 3.8281924083723253, 6.078747862615598, 2.1022283564016147, 5.316831086979245, 0.7445579751549053, 4.622656621744946, 1.8493085783016958, 6.237848148124421, 2.0219423235692116, 1.0044729120628748, 5.91955350667859, 0.97404471593805, 2.2350930660631056
Start iteration 1, value: 1.833604844203045, best: 1.833604844203045, dval: 1.0698744475848372
1.4850498457844585, 3.002173245168666, 1.384534466478679, 5.255709596227804, 5.985472374013028, 5.323246199920726, 1.430465636107199, 2.0320184059525572, 3.9014092334129646, 4.527525318620267, 3.686966707963977, 4.660108674602797, 1.8833116966015837, 2.0688059062972686, 3.832157856902801, 6.084195669810198, 2.1022283564016147, 5.318813035034857, 0.7525060864717755, 4.621965985

Start iteration 16, value: 0.008045626520221588, best: 0.008045626520221588, dval: 0.033470899817990056
Start iteration 17, value: 0.008853640784854764, best: 0.008045626520221588, dval: 0.050358652271402224
Big value increase: val: 0.008853640784854764, e.val: 0.008045626520221588, max(α): 20.83550910453891
1.1903007593531911, 2.415215281437068, 1.420676810892955, 5.230108955920492, 6.206766465908266, 4.763757994837148, 1.4814371219111373, 2.9415707594831813, 4.564659168992596, 5.101671715072845, 4.124708237187929, 4.073150710871193, 1.7106505852518576, 2.6524454953433825, 3.5257507903145124, 5.770030648784451, 2.1022283564016147, 5.756554564258813, 1.1391946791866991, 4.099303441243873, 3.1075697141849306, 6.756718559246838, 2.0219423235692116, 1.0025770198542294, 6.5693820355582595, 0.97404471593805, 2.1516088240913116
Start iteration 18, value: 0.005243476239461476, best: 0.005243476239461476, dval: 0.03169525189640438
Start iteration 19, value: 0.025145815899116377, best: 0.005243

Start iteration 35, value: 0.003449812479367688, best: 0.00041854746412834656, dval: 0.05564313949912786
Big value increase: val: 0.003449812479367688, e.val: 0.00041854746412834656, max(α): 33.45324908382997
1.20501823154221, 2.3649277572851735, 1.427439200936551, 5.210519066224171, 6.13810630357702, 4.808800771792458, 1.5504374421411369, 2.94820467894342, 4.5603154200592115, 5.100845097036531, 4.124515559594492, 4.022863186719352, 1.7391283469448708, 2.8257529334385194, 3.4139333994438967, 5.781759959705628, 2.1022283564016147, 5.756361886665377, 1.2351998181644581, 3.962052875363877, 3.1042162391277075, 6.737677697706676, 2.0219423235692116, 1.0430108461350716, 6.573223572139904, 0.97404471593805, 2.1516088158833284
Start iteration 36, value: 0.00037097384632373296, best: 0.00037097384632373296, dval: 0.0035024657374014417
Start iteration 37, value: 0.0003731322062552167, best: 0.00037097384632373296, dval: 0.00378640640072625
1.2049910472121685, 2.362841270076832, 1.427436707593228

1.2038740669301013, 2.3595145991247937, 1.4187276324145845, 5.199341584455451, 6.135359251489106, 4.8087149987793065, 1.56734770705643, 2.9497463609142542, 4.56378027616431, 5.101780054778086, 4.088542546001231, 4.018998375875168, 1.7293059650323066, 2.895175558541921, 3.4187844258952995, 5.793760820402599, 2.1022283564016147, 5.720388873071996, 1.2443869320869982, 3.941114563800297, 3.114124027997621, 6.738608923341029, 2.0219423235692116, 1.0252782352980117, 6.573014496453551, 0.97404471593805, 2.151608815883327
Start iteration 55, value: 0.00011579059256720234, best: 0.00011579059256720234, dval: 0.0014035690265256362
1.2038620337915034, 2.3593971450548272, 1.4187210996719486, 5.197647347813419, 6.134320559511847, 4.808674951478173, 1.568813641136519, 2.949747823843214, 4.5668265496339115, 5.102001486105006, 4.087606713262778, 4.018973994637609, 1.729293550246279, 2.901816792200403, 3.4187844454391034, 5.794261692423612, 2.1022283564016147, 5.719453040333538, 1.2443191848713122, 3.9

Start iteration 73, value: 3.698572998820007e-5, best: 3.698572998820007e-5, dval: 0.0007641731142778649
1.203366039684573, 2.3570330143904927, 1.4187194110990216, 5.190827498748876, 6.128559294351675, 4.795765733072777, 1.570944264543103, 2.9502443320832423, 4.584933806821362, 5.101149486668533, 4.075908442156331, 4.018527896700041, 1.7249034035871311, 2.939557073742343, 3.4228523083613243, 5.800274872884309, 2.1022283564016147, 5.707754769227072, 1.2423134185794962, 3.9409585252801316, 3.1147744914004014, 6.738560436457295, 2.0219423235692116, 0.9950823191059004, 6.57271963515072, 0.97404471593805, 2.151608815883326
Start iteration 74, value: 3.618495525850236e-5, best: 3.618495525850236e-5, dval: 0.000732403874894233
1.2032475664183964, 2.3570330665047448, 1.4187193853685918, 5.190842091877173, 6.128378211394866, 4.795028769787966, 1.5712696435885702, 2.9502863543966646, 4.5849426268895614, 5.1011425928724545, 4.0743085313536165, 4.018527907510428, 1.724944514607759, 2.9459130637654

...

Start iteration 125, value: 3.825991505665764e-6, best: 3.825991505665764e-6, dval: 0.000187603638828953
Start iteration 126, value: 3.921848371858656e-6, best: 3.825991505665764e-6, dval: 0.00034148980845043143
1.2031954537336564, 2.3583135070514816, 1.4186999965289395, 5.183490976491323, 6.116999675367165, 4.730897686178638, 1.574186600473006, 2.9506532195587383, 4.592217943619456, 5.100040205549828, 4.066136371113663, 4.018793645641853, 1.730924273571619, 2.984156022565789, 3.433549522235554, 5.809621060656292, 2.1022283564016147, 5.697982698184385, 1.2431044201850745, 3.9385527563350187, 3.1153848550966092, 6.738635178212854, 2.0219423235692116, 0.9673166326307264, 6.5727154873541895, 0.97404471593805, 2.1516088158833266
Start iteration 127, value: 3.7049206554326864e-6, best: 3.7049206554326864e-6, dval: 0.00021282850127241418
Start iteration 128, value: 5.161749822253997e-6, best: 3.7049206554326864e-6, dval: 0.0012457115911169159
Big value increase: val: 5.161749822253997e-6, e.

Start iteration 146, value: 2.6099545004586532e-6, best: 2.6099545004586532e-6, dval: 0.00021667628431487965
1.2031954901865718, 2.358400029198581, 1.4187286651258035, 5.183196714037536, 6.116457290453204, 4.715191087229221, 1.5749437455087563, 2.9506297484692197, 4.5922182221236225, 5.099745333607911, 4.065785102870868, 4.018815099886507, 1.732767620410246, 2.986963471019396, 3.434764502689721, 5.810527671932111, 2.1022283564016147, 5.69763142994159, 1.2432564665650654, 3.9383616021842833, 3.114961037781944, 6.738618669172167, 2.0219423235692116, 0.9665408812561755, 6.572697330806095, 0.97404471593805, 2.1516088158833266
Start iteration 147, value: 2.542648647462854e-6, best: 2.542648647462854e-6, dval: 0.00020239884042533296
Start iteration 148, value: 2.97123369055443e-6, best: 2.542648647462854e-6, dval: 0.0007095761099381384
Big value increase: val: 2.97123369055443e-6, e.val: 2.542648647462854e-6, max(α): 24.76826511205028
1.20319549459285, 2.358400140977091, 1.4186845660877487, 

Start iteration 166, value: 1.8363204536541594e-6, best: 1.8363204536541594e-6, dval: 0.0001773098602398382
Start iteration 167, value: 1.9494859061316904e-6, best: 1.8363204536541594e-6, dval: 0.0003520494447982823
1.2031956653695102, 2.3584020685267024, 1.4186851616703817, 5.181862623835869, 6.115231295906783, 4.704262735540799, 1.5753311177333074, 2.950740812022203, 4.594754264606427, 5.099260978720151, 4.066611880458758, 4.018826273372801, 1.7336203390790306, 2.988981755538929, 3.4360207446602, 5.812122800582783, 2.1022283564016147, 5.698458207529521, 1.2430739168197154, 3.9382143803064156, 3.1149154415579092, 6.738492967120693, 2.0219423235692116, 0.9665425998417622, 6.572629353956075, 0.97404471593805, 2.1516088158833266
Start iteration 168, value: 1.7120873358009236e-6, best: 1.7120873358009236e-6, dval: 0.0002113525370183722
1.203195684470489, 2.358402040201806, 1.4186851616797354, 5.1817467100005254, 6.11518389388628, 4.70441843656224, 1.5753353450298224, 2.9507444705142416, 4

Start iteration 187, value: 1.3062570998746555e-6, best: 1.2361292073350092e-6, dval: 0.0003688046805203661
1.2032226956888474, 2.3584018992383213, 1.4186851616862897, 5.180670537150817, 6.114695040079525, 4.695243395971895, 1.575766220827429, 2.950791758804151, 4.595601525792082, 5.09891376944155, 4.0666726507040725, 4.018620252569853, 1.7349247294411456, 2.990459910546149, 3.4370237342555696, 5.813880646311285, 2.1022283564016147, 5.698518977774835, 1.2431445245832953, 3.938214379834971, 3.1142914913723305, 6.738494492952783, 2.0219423235692116, 0.9665427464354436, 6.572627528245135, 0.97404471593805, 2.1516088158833266
Start iteration 188, value: 1.1915069606851965e-6, best: 1.1915069606851965e-6, dval: 0.0001525002952422784
1.2032250237432354, 2.358401893138114, 1.4186851616862937, 5.18065920668621, 6.114690669528554, 4.6948992308098445, 1.575765231009505, 2.95078473640595, 4.595601507231121, 5.098915528827402, 4.066673282177305, 4.018617459092986, 1.7350061354665112, 2.99047420878

(9.210281443498069e-7, Number[1.2032986209701937, 2.358401732835358, 1.4186851616863025, 5.180460939767263, 6.114562326809425, 4.68803456772096, 1.5758429266377212, 2.950845275768344, 4.595601520615441, 5.098718519442539  …  5.6986745391858005, 1.2431720091968603, 3.938214379835088, 3.1145793064785714, 0.45526054973003127, 2.0219423235692116, 0.9665428214975044, 6.572627528245022, 0.97404471593805, 2.1516088158833266], 197)

In [26]:
check_err_of_ansact(step5_start_params)

0.0005382235252532572